<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: simple_ppm.c

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the function `HLL_flux`. This function computes the right-hand side for $A_i$, excluding the gauge contribution.

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#HLL_flux): **The `HLL_flux` function**
1. [Step 3](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss the `HLL_flux` function for computing the right-hand side of $A_i$. The `calculate_phitilde_and_A_gauge_rhs` function (described [here](Tutorial-phitilde_and_A_gauge_rhs.ipynb)) provides the gauge contributions to the right-hand side, while this function calculates the contribution from $B$. For example,
$$
\partial_t A_y = \psi^{6} (v^z B^x - v^x B^z)
$$

<a id='HLL_flux'></a>

# Step 2: The `HLL_flux` function \[Back to [top](#toc)\]

$$\label{HLL_flux}$$


First, let's look at the arguments of `HLL_flux`:
```
double HLL_flux(const A_no_gauge_vars *restrict vars)
```

The only argument is a struct containing stencils for the needed input variables. This struct is defined as
```
typedef struct A_no_gauge_vars {
  double psi6;
  double B1r, B1l;
  double B2r, B2l;
  double c1_min, c1_max;
  double c2_min, c2_max;
  double v1rr, v1rl, v1lr, v1ll;
  double v2rr, v2rl, v2lr, v2ll;
} A_no_gauge_vars;
```
The 1 and 2 portions of the names come from the curl operation which gives the right-hand side. These $B^i$ are at the staggered grid points and are not cell-centered. The $r$ and $l$ represent the right/left faces. For example, $vxrr$ is the reconstructed right face of $v_x^R$, which is itself the reconstructed right face of $v_x$. The returned value is $A_i^{\rm RHS}$, where $i$ depends on the inputs.

Since filling this struct is less trivial than for most functions, we give a specific example assuming that we are calculating the $A_z$ term. In this case,
\begin{align}
v_1 &= v_x \\
v_2 &= v_y \\
B^1 &= B^x \\
B^2 &= B^y
\end{align}
and so on. That is to say, all the "1" variables are the $x$ components, and the "2" variables are the $y$ components.  For the point $(i,j,k)$, the $v$ variables come from the point $(i+1,j+1,k)$. In general, the directions perpendicular to the $i$ of $A_i^{\rm RHS}$ are increased by 1. The point of $B_1$ variables are from the point $(i,j+1,k)$, and the point of $B_2$ variables are from the point $(i+1,j,k)$. In general, the $B$ variables add 1 to the index $i$ of $A_i^{\rm RHS}$ permuted by 1. Finally, the element `psi6` stores $\psi^{6\phi}$ at the staggered point $(i+\frac{1}{2},j+\frac{1}{2},k)$. In general, `psi6` is staggered in the directions perpendicular to the $i$ of $A_i^{\rm RHS}$. Since $\psi^{6\phi}$ is naturally cell-centered, this usually requires interpolating the point before packing the struct. A table summarizing the grid points for these is given below:

|  $A_i^{\rm RHS}$ Direction  |  $v$ Index  | $B_1$ Index  | $B_2$ Index  | $psi6$ Index  |
|:-----------------:|:-------------:|:-----------:|:-----------:|:-----------------------------------:|
|         x         | $$(i,j+1,k+1)$$ | $$(i,j,k+1)$$ | $$(i,j+1,k)$$ | $$(i,j+\frac{1}{2},k+\frac{1}{2})$$ |
|         y         | $$(i+1,j,k+1)$$ | $$(i+1,j,k)$$ | $$(i,j,k+1)$$ | $$(i+\frac{1}{2},j,k+\frac{1}{2})$$ |
|         z         | $$(i+1,j+1,k)$$ | $$(i,j+1,k)$$ | $$(i+1,j,k)$$ | $$(i+\frac{1}{2},j+\frac{1}{2},k)$$ |

To see a practical example of how to pack this grid function, see `A_no_gauge_rhs.c` in the `GRHayL_IGM` implementation. This implementation also shows how to easily produce the desired indices depending on the $A_i^{\rm RHS}$ being computed.

Within the `HLL_flux` function, we first compute the face values for $A_3^{\rm RHS}$
$$
\left(A_3^{\rm RHS}\right)_{ij} = \psi^{6\phi} \left( \left(v_1\right)_{ij}\left(B_2\right)_{i} - \left(v_2\right)_{ij}\left(B_2\right)_{j} \right)
$$
where $i$,$j$ represent the faces $r$ and $l$. So, $\left(B_1\right)_{r}$ is the struct element `B1r`, and $\left(B_2\right)_{r}$ is the struct element `B2l`.

We also compute
$$
\Delta \tilde{B}_1 = \psi^{6\phi}\left( \left(B_1\right)_{r} - \left(B_1\right)_{l} \right) \\
\Delta \tilde{B}_2 = \psi^{6\phi}\left( \left(B_2\right)_{r} - \left(B_2\right)_{l} \right)
$$
and
$$
c_1^{\rm sum} = c_1^{\rm min} + c_1^{\rm max} \\
c_2^{\rm sum} = c_2^{\rm min} + c_2^{\rm max}
$$

With all of these, we can compute the RHS value using the HLL flux (see [see Del Zanna, Bucciantini & Londrillo A&A 400, 397 (2003), Eq. (44)])
$$
A_3^{\rm RHS} = \frac{c_1^{\rm max} c_2^{\rm max}}{c_1^{\rm sum} c_2^{\rm sum}}\left(A_3^{\rm RHS}\right)_{ll}
              + \frac{c_1^{\rm max} c_2^{\rm min}}{c_1^{\rm sum} c_2^{\rm sum}} \left(A_3^{\rm RHS}\right)_{lr}
              + \frac{c_1^{\rm min} c_2^{\rm max}}{c_1^{\rm sum} c_2^{\rm sum}} \left(A_3^{\rm RHS}\right)_{rl}
              + \frac{c_1^{\rm min} c_2^{\rm min}}{c_1^{\rm sum} c_2^{\rm sum}} \left(A_3^{\rm RHS}\right)_{rr}
              - \frac{c_1^{\rm min}c_1^{\rm max}\Delta \tilde{B}_2}{c_1^{\rm sum}}
              + \frac{c_2^{\rm min}c_2^{\rm max}\Delta \tilde{B}_1}{c_2^{\rm sum}}
$$

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log